# Attendance Report

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday
import glob

pd.set_option("display.max_columns", None)

# Type name of Month in 'X'

In [2]:
x = 'March'

In [3]:
df = pd.DataFrame(pd.date_range('2021-01-01', end='2021-12-31'), columns=['Date'])
df['DayName'] = pd.DatetimeIndex(df['Date']).day_name() # week day name
df['Month']   = pd.DatetimeIndex(df['Date']).month_name() # month

In [4]:
w = []
for i in df['DayName'] :
    if i == 'Sunday':
        w.append('Week off')
    elif i == 'Saturday':
        w.append('Week off')
    else:
        w.append('Working Day')
        
df['Week_off'] = w
        

In [5]:
class INTradingCalendar(AbstractHolidayCalendar):
    rules = [
         Holiday('New Year', month=1, day=1),
         Holiday('Republic Day', month=1, day=26),
         Holiday('Holi', month=3, day=29),
         Holiday('Good Friday', month=4, day=2),
         Holiday('Independance Day', month=8, day=15),
         Holiday('Raksha Bandhan', month=8, day=22),
         Holiday('Mahatma Gandhi Jayanti', month=10, day=2),
         Holiday('Dussehra', month=10, day=15),
         Holiday('Dewali', month=11, day=4),
         Holiday('Govardhan Puja', month=11, day=5),
         Holiday('Bhai Duj', month=11, day=6),
         Holiday('Christmas Day', month=12, day=25)
    ]


def get_trading_close_holidays(year):
    inst = INTradingCalendar()

    return inst.holidays(dt.datetime(year, 1, 1), dt.datetime(year, 12, 31), return_name= True)

In [6]:
holiday = pd.DataFrame(get_trading_close_holidays(2021))
holiday = holiday.reset_index().rename(columns={'index':'Date', 0:'Holiday'})
holiday= df.merge(holiday, how='left', on='Date')
holiday = holiday.replace('', np.nan)
holiday['Week_off'] = holiday['Holiday'].fillna(holiday['Week_off'])
holiday.drop(['Holiday'], axis = 1, inplace = True)

In [7]:
monthly_date=holiday.loc[holiday.Month==x]
monthly_date.drop(['Month'],axis= 1, inplace= True)

C:\Users\admin\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Import Data set of Attendace

In [8]:
file = glob.glob('*xls')

fd = pd.DataFrame()

for i in file:
    mn = pd.read_html(i)
    mn = pd.concat(mn)
    mn.columns = mn.columns.droplevel(level=1)
    mn.rename(columns={'##':'SNo'}, inplace= True)
    mn.columns = mn.columns.str.replace(' ', '')
    mn = mn[:-1]
    mn =mn[mn.AgentName.isin(['Sujata James','Shivangi Sharma', 'Bhawna', 'Shivani Godala','Anisha Yadav', 'Mehak Singh',
                              'Yawar Mallick','Sonia Satyan','Pawni Vaish'])]
  
    fd = fd.append(mn)

In [9]:
fd.head(10)

,SNo,AgentID,AgentName,Campaign,FirstLogin,LastLogout,LoginDuration,ModeDuration,OutboundCall,OutboundDuration,InboundCall,IBDuration,ManualCall,ManualDuration,IdleDuration,WrapupDuration,Transfer,TransferDuration,Conf.,Conf.Duration,Recall,RecallDuration,Hold,HoldDuration,Review,ReviewDuration,Sms,E-Mail,BreakDuration,Toilet_BreakDuration,First_BreakDuration,Second_BreakDuration,Supervisor_CallDuration,Talking_With_ManagerDuration,Team_Leader_SessionDuration,TotalBreakDuration,TotalTalkTime,TotalCalls,ATT,AHT
0,1,user15 (186),Sujata James,AUS,1st Mar 2021 06:26:20 AM,1st Mar 2021 03:01:49 PM,08:28:30,08:28:30,408.0,02:16:45,47.0,00:18:54,4.0,00:01:48,04:23:41,00:31:58,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:06:17,00:28:01,00:21:06,00:00:00,00:00:00,00:00:00,00:55:24,02:37:27,459.0,00:00:20,00:00:24
1,2,user2 (173),Shivangi Sharma,AUS,1st Mar 2021 06:29:12 AM,1st Mar 2021 03:02:25 PM,08:30:57,08:30:57,383.0,02:34:04,56.0,00:32:57,5.0,00:00:49,03:58:01,00:30:10,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:05:47,00:29:21,00:19:48,00:00:00,00:00:00,00:00:00,00:54:56,03:07:50,444.0,00:00:25,00:00:29
2,3,user6 (177),Mehak Singh,AUS,1st Mar 2021 06:32:10 AM,1st Mar 2021 03:01:51 PM,08:29:41,08:29:41,329.0,02:11:39,48.0,00:28:58,28.0,00:10:31,03:52:02,00:54:25,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:04:58,00:29:12,00:17:56,00:00:00,00:00:00,00:00:00,00:52:06,02:51:08,405.0,00:00:25,00:00:33
3,4,user14 (185),Yawar Mallick,AUS,1st Mar 2021 06:33:42 AM,1st Mar 2021 03:01:44 PM,08:28:02,08:28:02,294.0,01:58:22,45.0,00:25:33,39.0,00:24:03,03:25:51,01:06:33,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:17:44,00:28:44,00:21:12,00:00:00,00:00:00,00:00:00,01:07:40,02:47:58,378.0,00:00:26,00:00:37
4,5,user3 (174),Shivani Godala,AUS,1st Mar 2021 06:37:14 AM,1st Mar 2021 03:01:41 PM,08:06:29,08:04:52,226.0,01:42:32,30.0,00:15:24,40.0,00:38:22,03:16:41,01:15:46,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:07:28,00:29:35,00:19:04,00:00:00,00:00:00,00:00:00,00:56:07,02:36:18,296.0,00:00:31,00:00:47
5,6,user7 (178),Bhawna,AUS,1st Mar 2021 06:40:22 AM,1st Mar 2021 03:01:48 PM,08:17:51,08:17:51,355.0,02:41:37,41.0,00:23:09,4.0,00:00:59,03:29:36,00:41:35,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:12:24,00:29:30,00:19:01,00:00:00,00:00:00,00:00:00,01:00:55,03:05:45,400.0,00:00:27,00:00:34
11,12,user11(182),Anisha Yadav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,user12(183),Sonia Satyan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,14,user13(184),Pawni Vaish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,user7 (178),Bhawna,AUS,2nd Mar 2021 06:22:25 AM,2nd Mar 2021 09:34:58 AM,03:12:33,03:12:33,113.0,00:53:03,21.0,00:14:33,2.0,00:00:11,01:43:20,00:15:25,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,00:00:00,0.0,0.0,00:00:00,00:06:01,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:06:01,01:07:47,136.0,00:00:29,00:00:36


In [10]:
fd = fd.loc[:,['AgentName','FirstLogin','LastLogout','LoginDuration']]

In [11]:
fd['FirstLogin']= pd.to_datetime(fd['FirstLogin'])
fd['LastLogout'] = pd.to_datetime(fd['LastLogout'])

In [12]:
fd['LastLogout'] = fd['LastLogout'].dt.time

In [13]:
fd['Date']=fd['FirstLogin'].dt.date
fd['LoginTime']=fd['FirstLogin'].dt.time
fd.drop(['FirstLogin'], axis = 1 , inplace = True)
fd =fd[['AgentName', 'Date', 'LoginTime', 'LastLogout','LoginDuration']]
fd['Date'] = fd['Date'].fillna(method='ffill')

In [14]:
fd['Date'] = pd.to_datetime(fd['Date'])

In [15]:
fd.head(3)

,AgentName,Date,LoginTime,LastLogout,LoginDuration
0,Sujata James,2021-03-01,06:26:20,15:01:49,08:28:30
1,Shivangi Sharma,2021-03-01,06:29:12,15:02:25,08:30:57
2,Mehak Singh,2021-03-01,06:32:10,15:01:51,08:29:41


In [16]:
monthly_date = monthly_date.merge(fd, how='left', on='Date')

In [17]:
monthly_date.shape

(47, 7)

In [18]:
from datetime import time

In [19]:
e = pd.to_timedelta(time(hour=5 , minute=45, second=0).isoformat('auto'))
o = pd.to_timedelta(time(hour=6 , minute=45, second=0).isoformat('auto'))
l = pd.to_timedelta(time(hour=8 , minute=15, second=0).isoformat('auto'))
# For Absent Case
a = pd.to_timedelta(time(hour=0 , minute=0, second=0).isoformat('auto'))
# For Early Logout Case
hd = pd.to_timedelta(time(hour=4 , minute=45, second=0).isoformat('auto'))

In [20]:
monthly_date['LoginTime'].fillna(a, inplace=True)
monthly_date['LoginDuration'].fillna(a, inplace=True)
monthly_date['LastLogout'].fillna(a, inplace=True)

In [21]:
monthly_date['LastLogout'] = pd.to_timedelta(monthly_date['LastLogout'].astype(str))
monthly_date['LoginTime'] = pd.to_timedelta(monthly_date['LoginTime'].astype(str))
monthly_date['LoginDuration'] = pd.to_timedelta(monthly_date['LoginDuration'].astype(str))

In [22]:
r = []
for i in monthly_date.LoginTime:
    if i > e and i <=o:
        r.append('On Time')
    elif i > o and i <=l :
        r.append('Late')
    elif i == a:
        r.append('Absent')
    else:
        r.append('Half Day')
        
monthly_date['Remarks'] = r

# For Early Logout Case

In [23]:
r1 = []
for i in monthly_date.LoginDuration:
    if i <= hd:
        r1.append('Early Logout')
    else:
        r1.append('Logout')

monthly_date['Remarks_1'] = r1

In [24]:
monthly_date.loc[monthly_date['Week_off']=='Week off', 'Remarks'] = 'Week off'
monthly_date.loc[monthly_date['Week_off']=='Week off', 'Remarks_1'] = 'Week off'
monthly_date.loc[monthly_date['Remarks']=='Absent', 'Remarks_1'] = 'Absent'
monthly_date.loc[monthly_date['Remarks_1']=='Early Logout', 'Remarks'] = 'Half Day'

In [25]:
monthly_date.drop('Remarks_1', axis=1, inplace= True)

In [26]:
monthly_date['LoginTime'] = pd.to_datetime(monthly_date['LoginTime'],errors='ignore')
monthly_date['LoginDuration'] = pd.to_datetime(monthly_date['LoginDuration'],errors='ignore')
monthly_date['LastLogout'] = pd.to_datetime(monthly_date['LastLogout'],errors='ignore')

In [27]:
monthly_date.head(10)

,Date,DayName,Week_off,AgentName,LoginTime,LastLogout,LoginDuration,Remarks
0,2021-03-01,Monday,Working Day,Sujata James,0 days 06:26:20,0 days 15:01:49,0 days 08:28:30,On Time
1,2021-03-01,Monday,Working Day,Shivangi Sharma,0 days 06:29:12,0 days 15:02:25,0 days 08:30:57,On Time
2,2021-03-01,Monday,Working Day,Mehak Singh,0 days 06:32:10,0 days 15:01:51,0 days 08:29:41,On Time
3,2021-03-01,Monday,Working Day,Yawar Mallick,0 days 06:33:42,0 days 15:01:44,0 days 08:28:02,On Time
4,2021-03-01,Monday,Working Day,Shivani Godala,0 days 06:37:14,0 days 15:01:41,0 days 08:06:29,On Time
5,2021-03-01,Monday,Working Day,Bhawna,0 days 06:40:22,0 days 15:01:48,0 days 08:17:51,On Time
6,2021-03-01,Monday,Working Day,Anisha Yadav,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,Absent
7,2021-03-01,Monday,Working Day,Sonia Satyan,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,Absent
8,2021-03-01,Monday,Working Day,Pawni Vaish,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,Absent
9,2021-03-02,Tuesday,Working Day,Bhawna,0 days 06:22:25,0 days 09:34:58,0 days 03:12:33,Half Day


In [28]:
#monthly_date.to_csv('AttendanceReport.csv', index=False)